In [1]:
import feedparser
from pprint import pprint
from bs4 import BeautifulSoup
import pandas as pd

url = "http://news.google.com/news?q=covid&hl=en-US&sort=date&gl=US&num=10&output=rss"

class ParseFeed():
    def __init__(self, url):
        self.feed_url = url
        
    def clean(self, html):
        '''
        Get the text from html and do some cleaning
        '''
        soup = BeautifulSoup(html)
        text = soup.get_text()
        text = text.replace('\xa0', ' ')
        return text

    def parse(self):
        newsDescription = []
        newsUrl = []
        newsDate = []
        
        '''
        Parse the URL, and print all the details of the news 
        '''        
        feeds = feedparser.parse(self.feed_url).entries
        
        for f in feeds:
            newsDescription.append(self.clean(f.get("description", "")))
            newsUrl.append(f.get("link", ""))
            newsDate.append(f.get("published", ""))
            
        df = pd.DataFrame({'Description':newsDescription, 'URL':newsUrl, 'Date Published':newsDate})            
        return df
    
htmlOutput = ""
feed = ParseFeed(url)
df = feed.parse()

In [2]:
df['Date'] = pd.to_datetime(df['Date Published'])
df = df.head(10).sort_values(by = ['Date'], ascending=False)

# Saving data to html output
htmlOutput = """<html><title>COVID-19 Daily News</title><body><style>
		body {
		  background-image: url('background.jpg');
		  background-repeat: no-repeat;
		  background-attachment: fixed;
		  background-size: cover;
		}
	  </style><center><h1>COVID-19 Daily News</h1></center><dl>"""

for index,row in df.iterrows():
    htmlOutput = htmlOutput + "<dt>" + row['Description'] + "</dt><dd>" + row['Date Published'] + "</dd><dd><a href =" + row['URL'] + ">" + row['URL'] + "</a></dd><br>"
htmlOutput = htmlOutput + "</dl></body></html>"
text_file = open("home.html", "w")
text_file.write(htmlOutput)
text_file.close()

df.to_csv('data.csv', index=False)
df

,Description,URL,Date Published,Date
3,"Coronavirus (COVID-19) Update: November 3, 202...",https://www.fda.gov/news-events/press-announce...,"Tue, 03 Nov 2020 22:30:00 GMT",2020-11-03 22:30:00+00:00
4,Tracking COVID-19 in Alaska: 382 new cases rep...,https://www.adn.com/alaska-news/2020/11/03/tra...,"Tue, 03 Nov 2020 21:25:40 GMT",2020-11-03 21:25:40+00:00
6,Oregon surpasses 700 COVID-19 deaths KGW.com,https://www.kgw.com/article/news/health/corona...,"Tue, 03 Nov 2020 21:01:00 GMT",2020-11-03 21:01:00+00:00
1,Governor Cuomo Updates New Yorkers on State's ...,https://www.governor.ny.gov/news/governor-cuom...,"Tue, 03 Nov 2020 20:57:56 GMT",2020-11-03 20:57:56+00:00
0,COVID-19 news from across the United States C...,https://www.cbsnews.com/live-updates/covid-19-...,"Tue, 03 Nov 2020 20:01:00 GMT",2020-11-03 20:01:00+00:00
8,Recon: Brazil's Anvisa allows J&J to resume CO...,https://www.raps.org/news-and-articles/news-ar...,"Tue, 03 Nov 2020 17:53:27 GMT",2020-11-03 17:53:27+00:00
9,"On Election Day, Maine sees record 127 cases o...",https://www.pressherald.com/2020/11/03/on-elec...,"Tue, 03 Nov 2020 15:10:00 GMT",2020-11-03 15:10:00+00:00
2,COVID-19 Daily Update 11-3-2020 West Virginia...,https://dhhr.wv.gov/News/2020/Pages/COVID-19-D...,"Tue, 03 Nov 2020 15:07:22 GMT",2020-11-03 15:07:22+00:00
7,Latest on COVID-19 in MN: Daily cases skyrocke...,https://www.mprnews.org/story/2020/11/03/lates...,"Tue, 03 Nov 2020 11:00:00 GMT",2020-11-03 11:00:00+00:00
5,COVID-19 Daily Update 11-2-2020 West Virginia...,https://dhhr.wv.gov/News/2020/Pages/COVID-19-D...,"Mon, 02 Nov 2020 15:05:46 GMT",2020-11-02 15:05:46+00:00
